<a href="https://colab.research.google.com/github/cbannard/compling/blob/main/seq2seq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
%reset
import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader
from collections import Counter

import torchtext
from torchtext.legacy.datasets import Multi30k
from torchtext.legacy.data import Field, BucketIterator
from torchtext.legacy.vocab import Vocab
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
import spacy
import numpy as np

import random
import math
import time
import platform
import os
import io
import copy
global enc, dec, model
TFR = 0.5
data_path = path = F"/content/gdrive/My Drive/Modelling_Sentence_Repetition/Code/s2s/" 
train_filepaths = [data_path + 'source-all.txt']
val_filepaths = train_filepaths
test_filepaths = train_filepaths


Once deleted, variables cannot be recovered. Proceed (y/[n])? y
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [23]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True


In [24]:
def strip_string(str):
    out = ''
    for char in str:
        if char not in ['“', '”', '[', ']', '<', '>']:
            out = out + char
    return out

In [25]:
def get_riches_targets():
    max_len = 0
    print('getting Riches data')
    outlist = []
    with open( data_path + 'target-lines-Riches-punct.txt', encoding = 'utf-8') as f:
        inlist = f.read().split('\n')
    for item in inlist:
        index = item.find(',')
        if index > 0:
            outlist.append(item[index+1:].split())
    for item in outlist:
        if len(item) > max_len:
            max_len = len(item)
    print('max length is', max_len)
    return outlist

test_set = get_riches_targets()

src_counts = {}

getting Riches data
max length is 14


In [26]:
def build_vocab(filepath, test_set):
    counter = Counter()
    with open(filepath, encoding="utf8") as infile:
        inlist = infile.read().splitlines()
        print(len(inlist))
        #for string_ in ['<pad>', '<bos>', '<eos>']:
        #    for i in range(100):
        #        counter.update(string_)
        for line in inlist:
            line = line.split()
            for string_ in line:
                ##print(string_)
                ## just pass [string] instead of tokenizer
                counter.update([strip_string(string_)])
        for line in test_set:
            for i in range(100):
                for string_ in line:
                    ##print(string_)
                    counter.update(strip_string(string_))
    return Vocab(counter, specials=['<unk>', '<bos>', '<eos>', '<pad>'])


In [27]:
full_vocab = build_vocab(test_filepaths[0], test_set)
src_vocab = Vocab(counter=full_vocab.freqs, min_freq=5, specials=['<unk>', '<bos>', '<eos>', '<pad>'])
#trg_vocab = build_vocab(train_filepaths[1], trg_tokenizer)
trg_vocab = src_vocab
del(full_vocab)

embedding_length = 50
temp_matrix = {}
embeddings_matrix = torch.zeros((len(src_vocab), 50))
PAD_IDX = trg_vocab.stoi['<pad>']
FUNCTION_WORDS = ['a', 'an', 'the',
                  'and', 'but', 'with', 'of', 'not', 'as', 'with', 'at', 'more',
                  'in', 'on', 'out', 'up', 'of', 'off', 'here', 'there',
                 'I', 'you', 'we', 'they', 'me', 'us', 'them', 'mine',
                  'yours', 'ours', 'theirs',
                  'he', 'she', 'it', 'this', 'that', 'these', 'those', 'him', 'her',
                  'is', 'are', 'be', 'was', 'were', "I'm", "you're", "he's", "she's",
                  "it's", "we're", "they're", "that's", "here's", "there's",
                  'have', 'can', 'will', 'had', 'could', 'would', "can't", "won't",
                  'do', "don't", 'does'
                                 'what', 'where', 'which', 'who', 'how']

PRETRAINED = 'False'
HID_DIM = 100
N_LAYERS = 1
ENC_DROPOUT = 0.0
DEC_DROPOUT = 0.0
INPUT_DIM = len(src_vocab)
OUTPUT_DIM = len(trg_vocab)
ENC_EMB_DIM = 50
DEC_EMB_DIM = 50
N_EPOCHS = 2
EPOCH_SIZE = 10


1015640


In [28]:
#train_data, valid_data, test_data = Multi30k.splits(exts = ('.de', '.en'), 
#                                                  fields = (SRC, TRG))
def data_process(filepaths):
    MAX_LEN = 15
    raw_src_iter = iter(io.open(filepaths[0], encoding="utf8"))
    raw_trg_iter = iter(io.open(filepaths[0], encoding="utf8"))
    data = []
    for (raw_src, raw_trg) in zip(raw_src_iter, raw_trg_iter):
        raw_src = raw_src.split()
        for i in range(len(raw_src)):
            raw_src[i] = strip_string(raw_src[i])
        if  len(raw_src) < MAX_LEN:
            #raw_src = ['<bos>'] + raw_src + ['<eos>']
            #while len(raw_src) < MAX_LEN + 2:
            ##    raw_src = ['<pad>'] + raw_src
            #    raw_src = raw_src + ['<pad>']
            #raw_src = ' '.join(raw_src)
            #print(raw_src)
            for i in range(len(raw_src)):
                ##raw_src[i] = strip_string(raw_src[i])
                if src_vocab.stoi[raw_src[i]] == 0:
                    #print('unk', raw_src[i])
                    int = random.randint(4, len(src_vocab) -1)
                    #print(int)
                    #print('random', src_vocab.itos[int])
                    raw_src[i] = src_vocab.itos[int]
            raw_trg = copy.deepcopy(raw_src)
            #raw_src = ['<bos>'] + raw_src + ['<eos>']
            raw_trg = ['<bos>'] + raw_trg + ['<eos>']
            while len(raw_src) < MAX_LEN:
                #raw_src = raw_src + ['<pad>']
                raw_src = ['<pad>'] + raw_src
                raw_trg = raw_trg +  ['<pad>']
            #src_tensor_ = torch.tensor([src_vocab[token] for token in raw_src],
            #                            #src_tokenizer(raw_src)],
            #    dtype=torch.long)
            src_tensor_ = torch.tensor([src_vocab[token] for token in raw_src], dtype=torch.long)
            #print(len(src_tensor_))
            if src_tensor_[-1] == src_vocab.stoi['\n']:
                src_tensor_ = src_tensor_[0:-1]
            trg_tensor_ = torch.tensor([src_vocab[token] for token in raw_trg], dtype=torch.long)
        #print(len(src_tensor_))
            if trg_tensor_[-1] == src_vocab.stoi['\n']:
                trg_tensor_ = src_tensor_[0:-1]
            #trg_tensor_ = torch.tensor([src_vocab[token] for token in src_tokenizer(raw_trg)],
            #                           dtype=torch.long)
            #if trg_tensor_[-1] == src_vocab.stoi['\n']:
            #    trg_tensor_ = trg_tensor_[0:-1]
            data.append((src_tensor_, trg_tensor_))
    return data
    
train_data = data_process(train_filepaths)
val_data = data_process(val_filepaths)
test_data = data_process(test_filepaths)

In [29]:
def load_embs():
    global enc, dec, model
    print_emb('wants')
    for file in ['enc-embs-1000.out', 'dec-embs-1000.out']:
        with open(data_path + file, 'r', encoding = 'utf-8') as enc_file:
            inlist = enc_file.read().splitlines()
            print(len(inlist))
            for line in inlist:
                line = line.split(',')
                word, emb = line[0], ' '.join(line[1:])
                #print(word,emb)
                if word not in FUNCTION_WORDS:
                    emb = torch.tensor(np.fromstring(emb, sep = ' '))
                    if 'enc-embs' in file:
                        int = src_vocab.stoi[word]
                        #print(word, len(word), int)
                        if word == 'wants':
                            print('wants', enc.embedding.weight.data[int])
                        enc.embedding.weight.data[int] = emb
                        if word == 'wants':
                            print('enc', enc.embedding.weight.data[int])
                    elif 'dec-embs' in file:
                        ##print('dec')
                        int = trg_vocab.stoi[word]
                        dec.embedding.weight.data[int] = emb
    print_emb('wants')
    print_emb('is')

def print_emb(word):
    global enc
    lookup = torch.tensor([src_vocab.stoi[word]], dtype=torch.long)
    emb = enc.embedding.weight.data[lookup]
    print(emb)

In [30]:
def generate_batch(data_batch):
    #print('len in', len(data_batch))
    #print(data_batch)
    src_batch, trg_batch = [], []
    for (src_item, trg_item) in data_batch:
        ##### I think we only want to add beg and end markers to the target, not to the source...
        #src_batch.append(src_item)
        #trg_batch.append(trg_item)
        src_batch.append(torch.cat([src_item], dim=0))
        trg_batch.append(torch.cat([trg_item], dim=0))
        #src_batch.append(torch.cat([torch.tensor([BOS_IDX]), src_item, torch.tensor([EOS_IDX])], dim=0))
        #trg_batch.append(torch.cat([torch.tensor([BOS_IDX]), trg_item, torch.tensor([EOS_IDX])], dim=0))
    src_batch = pad_sequence(src_batch, padding_value=PAD_IDX)
    trg_batch = pad_sequence(trg_batch, padding_value=PAD_IDX)
    #print('len out', len(src_batch))
    #print(src_batch)
    return src_batch, trg_batch


In [31]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [32]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, n_layers, dropout, weights_matrix):
        super().__init__()

        self.hid_dim = hid_dim
        self.n_layers = n_layers
        if PRETRAINED == 'False':
            self.embedding = nn.Embedding(input_dim, emb_dim, padding_idx= PAD_IDX)
        elif PRETRAINED == 'True':
            print('encoder using trained embeddings')
            self.embedding = nn.Embedding.from_pretrained(weights_matrix, freeze = 'False',padding_idx = PAD_IDX)

        ##self.embedding = create_emb_layer(weights_matrix)
        lookup = torch.tensor([src_vocab.stoi['baby']], dtype=torch.long)
        emb = self.embedding(lookup)
        #print(emb)
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout = dropout)

        self.dropout = nn.Dropout(dropout)

    def forward(self, src):

        #src = [src len, batch size]

        embedded = self.dropout(self.embedding(src))
        #print('done')
        #print(embedded[209])
        #embedded = [src len, batch size, emb dim]

        outputs, (hidden, cell) = self.rnn(embedded)

        #outputs = [src len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]

        #outputs are always from the top hidden layer
        ##print('cell', cell[0][0])
        ## add random noise (mean 0, var 1) to the cell state
        rndm = torch.randn(cell.size()).to(device)
        #cell = cell + (0.5 * rndm)
        #hidden = hidden + (0.5 * rndm)
        #print('rand', rndm[0][0])
        #print('new', cell[0][0])
        #for name in Encoder.named_parameters(self):
        #    if 'weight' in name[0]:
        #        print('weights', name)
        return hidden, cell


In [33]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, n_layers, dropout, weights_matrix):
        super().__init__()

        self.output_dim = output_dim
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        if PRETRAINED == 'False':
            self.embedding = nn.Embedding(output_dim, emb_dim, padding_idx = PAD_IDX)
        elif PRETRAINED == 'True':
            print('decoder using trained embeddings')
            self.embedding = nn.Embedding.from_pretrained(weights_matrix, freeze = 'False',
                                                         padding_idx= PAD_IDX)
        ##self.embedding = nn.Embedding(output_dim, emb_dim)
        ##self.embedding = nn.Embedding.from_pretrained(weights_matrix, freeze = 'False')
        lookup = torch.tensor([src_vocab.stoi['baby']], dtype=torch.long)
        emb = self.embedding(lookup)
        ##print(emb)
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout = dropout)

        self.fc_out = nn.Linear(hid_dim, output_dim)

        self.dropout = nn.Dropout(dropout)

    def forward(self, input, hidden, cell):

        #input = [batch size]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]

        #n directions in the decoder will both always be 1, therefore:
        #hidden = [n layers, batch size, hid dim]
        #context = [n layers, batch size, hid dim]

        input = input.unsqueeze(0)

        #input = [1, batch size]

        embedded = self.dropout(self.embedding(input))

        #embedded = [1, batch size, emb dim]

        output, (hidden, cell) = self.rnn(embedded, (hidden, cell))
        #print('out', output)
        #print('hid', hidden)
        #output = [seq len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        rndm = torch.randn(cell.size()).to(device)
        #cell = cell + (0.25 * rndm)
        #hidden = hidden + (0.25 * rndm)
        #seq len and n directions will always be 1 in the decoder, therefore:
        #output = [1, batch size, hid dim]
        #hidden = [n layers, batch size, hid dim]
        #cell = [n layers, batch size, hid dim]

        prediction = self.fc_out(output.squeeze(0))

        #prediction = [batch size, output dim]

        return prediction, hidden, cell


In [34]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()

        self.encoder = encoder
        self.decoder = decoder
        self.device = device

        assert encoder.hid_dim == decoder.hid_dim, \
            "Hidden dimensions of encoder and decoder must be equal!"
        assert encoder.n_layers == decoder.n_layers, \
            "Encoder and decoder must have equal number of layers!"

    def forward(self, src, trg, teacher_forcing_ratio = 0.5):
        #print("TFR =", teacher_forcing_ratio)
        #print('fw src', src)
        #src = [src len, batch size]
        #trg = [trg len, batch size]
        #teacher_forcing_ratio is probability to use teacher forcing
        #e.g. if teacher_forcing_ratio is 0.75 we use ground-truth inputs 75% of the time

        batch_size = trg.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim

        #tensor to store decoder outputs
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)

        #last hidden state of the encoder is used as the initial hidden state of the decoder
        hidden, cell = self.encoder(src)

        #first input to the decoder is the <sos> tokens
        input = trg[0,:]

        for t in range(1, trg_len):

            #insert input token embedding, previous hidden and previous cell states
            #receive output tensor (predictions) and new hidden and cell states
            output, hidden, cell = self.decoder(input, hidden, cell)

            #place predictions in a tensor holding predictions for each token
            outputs[t] = output

            #decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            #print('tf', teacher_force)
            #get the highest predicted token from our predictions
            top1 = output.argmax(1)

            #if teacher forcing, use actual next token as next input
            #if not, use predicted token
            input = trg[t] if teacher_force else top1

        return outputs

In [35]:
criterion = nn.CrossEntropyLoss(ignore_index=PAD_IDX)

def train(model: nn.Module,
          iterator: torch.utils.data.DataLoader,
          optimizer: optim.Optimizer,
          criterion: nn.Module,
          clip: float):

    model.train()

    epoch_loss = 0
    for _, (src, trg) in enumerate(iterator):
        ##print('src', src)
        src, trg = src.to(device), trg.to(device)

        optimizer.zero_grad()

        output = model(src, trg)

        output = output[1:].view(-1, output.shape[-1])
        trg = trg[1:].view(-1)

        loss = criterion(output, trg)

        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)

        optimizer.step()

        epoch_loss += loss.item()
    return epoch_loss / len(iterator)

In [36]:
def evaluate(model: nn.Module,
             iterator: torch.utils.data.DataLoader,
             criterion: nn.Module):

    model.eval()

    epoch_loss = 0

    with torch.no_grad():

        for _, (src, trg) in enumerate(iterator):
            src, trg = src.to(device), trg.to(device)
            output = model(src, trg, 0) #turn off teacher forcing

            output = output[1:].view(-1, output.shape[-1])
            trg = trg[1:].view(-1)

            loss = criterion(output, trg)

            epoch_loss += loss.item()
    return epoch_loss / len(iterator)

In [37]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [38]:
def test_manc():
    test_list = []
    count = 0
    test_list = ["what do you want the thick pens"]
    #, "she was quite happily pulling it", "<unk> these <unk> <unk> are these your water wings", "do you think #the policeman'll be after him", "that's <unk> that's food for the cat", "is it a big hat that", "are you #building it up or breaking it up", "there were two on there earlier , weren't there", "well I can't throw it #now , can I", "and what does a cow say"]
    #while len(test_list) < 10:
    #    line = tensor_to_line(random.choice(train_data))
    #    print(line)
    #    if (len(line) > 5
    #        and len(line) <10):
    #        test_list.append(' '.join(line))
    for line in test_list:
        print(line)
        output = translate_sentence(line, model, 'cuda', 50)
        print('input', line)
        print('output', output)
        count += len(output)
    print('word count', count)

In [ ]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)
        
model.apply(init_weights)

In [39]:
out_dir = data_path + "saved-models/"

def count_parameters(model: nn.Module):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

def run_model(name_string):
    print(f'The model has {count_parameters(model):,} trainable parameters')
    CLIP = 1
    print('out_dir', out_dir)
    print('hid_dim', HID_DIM)
    print('TFR', TFR)
    print('epochs', N_EPOCHS)
    best_valid_loss = float('inf')
    MAX_LEN = 20
    max_target = 200
    name_string = name_string + str(HID_DIM) + '-50-ep-'
    print('name_string', name_string)
    for epoch in range(1, N_EPOCHS +1):
        part_tr, part_va = [], []
        while len(part_tr) < 8000:
            part_tr.append(random.choice(train_data))
        while len(part_va) < 2000:
            part_va.append(random.choice(train_data))
        print('tr', part_tr[0])
        print('va', part_va[0])
        #for i in range(len(part_tr)):
        #    if len(part_tr[i][1]) > max_target:
        #        part_tr[i] = [part_tr[i][0], truncate_random(part_tr[i][1], max=max_target)]
        #        ##part_tr[i][1] = truncate_random(part_tr[i][1], max_target)
        #        #print('done')
        #        #print('in, out', part_tr[i][0], part_tr[i][1])
        #train_iter = DataLoader(part_tr, batch_size=args.batch_size, shuffle=True, num_workers=4, **kwargs)
        train_iter = DataLoader(part_tr, batch_size=BATCH_SIZE,
                                shuffle=True, collate_fn=generate_batch)
        valid_iter = DataLoader(part_va, batch_size=BATCH_SIZE,
                                shuffle=True, collate_fn=generate_batch)
        #test_iter = DataLoader(test_data, batch_size=BATCH_SIZE,
        #                       shuffle=True, collate_fn=generate_batch)


        start_time = time.time()
        train_loss = train(model, train_iter, optimizer, criterion, CLIP)
        valid_loss = evaluate(model, valid_iter, criterion)
        del(part_tr)
        del(part_va)
        del(train_iter)
        del(valid_iter)
        end_time = time.time()

        epoch_mins, epoch_secs = epoch_time(start_time, end_time)
        ##print('epoch', epoch+1, epoch_mins, epoch_secs)
        print(f'Epoch: {epoch:02} | Time: {epoch_mins}m {epoch_secs}s')
        print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
        print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')
        torch.cuda.empty_cache()
        if (epoch > 0
                and epoch % 10 == 0):
            test_riches(name_string + str(epoch))
            test_manc()
            torch.save(model.state_dict(), out_dir + name_string + str(epoch) + '.pth')
        #if (epoch > 0
        #            and epoch % 50 == 0):
        #    add_noise()

N_EPOCHS = 2

PRETRAINED = 'False'
for HID_DIM in [500]:
    #[250, 500]:
        for i in range(1,6):
            BATCH_SIZE = 10
            print('TFR=', TFR)
            print('run is', str(i))
            name_string = 'pt-tf05-500-run-' + str(i) +'-wemb-06-07-'
            enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, N_LAYERS, ENC_DROPOUT, embeddings_matrix)
            ##enc.embedding.weight.data = embeddings_matrix
            dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, N_LAYERS, DEC_DROPOUT, embeddings_matrix)
            model = Seq2Seq(enc, dec, device).to(device)
            model.apply(init_weights)
            optimizer = optim.Adam(model.parameters())
            load_embs()
            print(model.eval())
            run_model(name_string)


TFR= 0.5
run is 1


NameError: ignored

In [ ]:
def translate_sentence(sentence = "there were two on there earlier , weren't there", model = model, device = 'cuda', max_len = 50):
    ##sentence = "there goes a man"
    #print("INPUT:", sentence)
    ##sentence = 'hoe gaat het nu met je'
    ##print('sentence', sentence)
    #if isinstance(sentence, str):
    #    sentence = '<bos> ' + sentence + ' <eos>'
    #    #sentence = sentence.split()
    #else:
    #    sentence = ['<bos>' + sentence + '<eos']
    #    #sentence = sentence.split()
    #pad out to max length + 2
    sentence = sentence.split()
    while len(sentence) < 15:
    #    ## pad at right. This is what pad sequences does.
    #    sentence = sentence + ['<pad>']
       sentence = ['<pad>'] + sentence
    tokens = [token.lower() for token in sentence]
    #print(tokens)
    src_indexes = [src_vocab.stoi[token] for token in tokens]
    src_tensor = torch.LongTensor(src_indexes).unsqueeze(1).to(device)
    ##print(src_indexes)
    #print(src_tensor)
    src_len = torch.LongTensor([len(src_indexes)])
    with torch.no_grad():
        hidden, cell = model.encoder(src_tensor)
    #print('hidden', hidden)
    #print('cell', cell)
    ##mask = model.create_mask(src_tensor)
    trg_indexes = [trg_vocab.stoi['<bos>']]
    output_indexes = []
    #attentions = torch.zeros(max_len, 1, len(src_indexes)).to(device)
    ##cell = torch.zeros(2,1,512).to(device)
    for i in range(max_len):
        trg_tensor = torch.LongTensor([trg_indexes[-1]]).to(device)
        #print('trg_tensor', trg_tensor)
        with torch.no_grad():
            output, hidden, cell = model.decoder(trg_tensor, hidden.to(device), cell.to(device))
        #print('out', output, len(output[0]))
        res, ind = output.topk(3)

        m = nn.Softmax(dim=1)
        temp = m(output)
        prob, ind2 = temp.topk(3)
        prop = prob[0][0]
        ##print(prob)
        ##print(res, len(res), ind)
        #print(trg_vocab.itos[ind[0][0].item()], trg_vocab.itos[ind[0][1].item()], trg_vocab.itos[ind[0][2].item()])
        #print('activations', round(res[0][0].item(),2), round(res[0][1].item(),2),
        #      round(res[0][2].item(),2))
        #prop = round(res[0][0].item()/(res[0][1].item() + res[0][2].item()), 2)
        #if trg_vocab.itos[ind[0][0]] in FUNCTION_WORDS:
        #    print('fword')
        if (trg_vocab.itos[ind[0][0]] in FUNCTION_WORDS
            and prop > 0.70):
            #print('adding fword')
            pred_token = output.argmax(1).item()
        elif (trg_vocab.itos[ind[0][0]] not in FUNCTION_WORDS
            and prop > 0.6):
            pred_token = output.argmax(1).item()
        elif trg_vocab.itos[output.argmax(1).item()] == '<eos>':
            pred_token = output.argmax(1).item()
        else:
            pred_token = trg_vocab.stoi['<pad>']

        ##pred_token = output.argmax(1).item()
        ###here we feed in the actual model prediction, rather than the padding token'
        ### but we collect the padding token for production
        trg_indexes.append(output.argmax(1).item())
        #trg_indexes.append(pred_token)
        if trg_vocab.itos[pred_token] != '<bos>':
            output_indexes.append(pred_token)

        #print('target word =', sentence[i+1])
        #print('candidates', trg_vocab.itos[ind[0][0]], trg_vocab.itos[ind[0][1]], trg_vocab.itos[ind[0][2]])
        #print('prop', prop)
        #print('pred', trg_vocab.itos[pred_token], pred_token)
        #print('feeding back', trg_indexes)
        #print('feeding', [trg_vocab.itos[i] for i in trg_indexes])
        #print('pred', pred_token, trg_vocab.itos[pred_token])
        #print('actual pred', output.argmax(1).item(), trg_vocab.itos[output.argmax(1).item()])

        #print('feeding', [trg_vocab.itos[i] for i in trg_indexes])
        #print('output indexes', output_indexes)
        if pred_token == trg_vocab.stoi['<eos>']:
            break
    #print('getting output', output_indexes)
    output_tokens = [trg_vocab.itos[i] for i in output_indexes]
    #print("OUTPUT", trg_tokens[1:])
    ##print('output', output_tokens)
    return(output_tokens)



